# Technique - Divide and Conquer

- **Divide** into number of subproblems that are smaller instances of same problem
- **Conquer** subproblems by solving them recursively; if they're small enough, they can be solved straightforwardly.
- **Combine** solutions to solve original problem

## Example: Mergesort

Mergesort is the classical example of a divide and conquer algorithm. We divide a non-empty array into subarrays to sort; a single element array is considered sorted. Then we do a linear-time merging operation to put them into sorted order.

In [ ]:
from copy import deepcopy
from random import randint

def merge(arr1, arr2):
    merged = []
    i = j = 0

    while i < len(arr1) and j < len(arr2):
        if arr1[i] <= arr2[j]:
            merged.append(arr1[i])
            i += 1
        else:
            merged.append(arr2[j])
            j += 1
    return merged + arr1[i:] + arr2[j:]
        
def mergesort(arr, lo, hi):
    if lo == hi:
        return [arr[lo]]
    
    mid = (hi+lo)//2
    arr1 = mergesort(arr,lo,mid)
    arr2 = mergesort(arr,mid+1,hi)
    return merge(arr1, arr2)
    

for test_len in [1,100,101]:
    unsorted_arr = [randint(-1000,1000) for i in range(test_len)]
    unsorted_copy = deepcopy(unsorted_arr)
    assert sorted(unsorted_copy) == mergesort(unsorted_arr, 0, len(unsorted_arr)-1)


## Example: [Maximum Subarray](https://leetcode.com/problems/maximum-subarray/), [Best Time to Buy and Sell Stock](https://leetcode.com/problems/best-time-to-buy-and-sell-stock/)

"Best Time to Buy and Sell Stock" becomes the Maximum Subarray problem when considering the array of daily changes in the stock price. The following implementation comes from CLRS 3rd ed. (p 71 and 73). Since the maximum subarray for any array is either found entirely left of mid, entirely right of mid, or crossing the mid, we do the following:

- **Divide** - Examine three subarrays of the current array: the left from the mid, right from the mid, and crossing mid.
- **Conquer** - Recurse into left and right until we hit the base case. Use a special O(n) procedure for calculating the sum of the crossing subarray.
- **Combine** - Since each recursive step returns the sum and indices for the max subarray, pick whichever one is the greatest. 

```
crossing_mid_procedure(arr, lo, mid, hi):
   starting at mid and going to lo:
     find biggest left subarr using running sum and its index
   starting at mid+1 and going to hi:
      find biggest right subarr using running sum and its index
   return index for biggest left, index for biggest right, and their combined sums

max_subarr(arr, lo, hi):
   // Base case
   if only one element in arr:
      return (lo, hi, val of the element)
   
   // Recursion
   get indices/amount of biggest subarr in bottom half 
   get indices/amount of biggest subarr in top half
   get indices/amount of biggest subarr crossing mid using above procedure
   
   return indices/amount of biggest of three above subarrs
```

In [ ]:
def find_max_crossing_subarray(arr, lo, mid, hi):
    l_sum = r_sum = -float('inf')
    l_csum = r_csum = 0
    for l in range(mid,lo-1,-1):
        l_csum += arr[l] 
        max_left = l if l_csum > l_sum else max_left
        l_sum = max(l_sum, l_csum)

    for r in range(mid+1,hi+1):
        r_csum += arr[r] 
        max_right = r if r_csum > r_sum else max_right
        r_sum = max(r_sum, r_csum)
    return (max_left, max_right, l_sum + r_sum)
    
def find_maximum_subarray(arr, lo, hi):
    if hi == lo:
        return (lo, hi, arr[lo])

    mid = (hi + lo) // 2
    l_lo, l_hi, l_sum = find_maximum_subarray(arr, lo, mid)
    r_lo, r_hi, r_sum = find_maximum_subarray(arr, mid+1, hi)
    c_lo, c_hi, c_sum = find_max_crossing_subarray(arr, lo, mid, hi)

    best = max(l_sum, r_sum, c_sum)
    if best == l_sum:
        return (l_lo, l_hi, l_sum)
    elif best == r_sum:
        return (r_lo, r_hi, r_sum)
    return (c_lo, c_hi, c_sum)


class Solution:
    def maxSubArray(self, arr):
        return find_maximum_subarray(arr, 0, len(arr)-1)[2]
    
s = Solution()
cases = [
    ([-2, 1, -3, 4, -1, 2, 1, -5, 4], 6),
    ([1], 1),
    ([0], 0),
    ([-1], -1),
    ([-2147483647], -2147483647),
    ([-3, -2, -1], -1),
    ([-3, -2, 1], 1)

]
for arr, expected in cases:
    actual = s.maxSubArray(arr)
    assert expected == actual, f"{arr}, {expected} != {actual}"


## Example: [Majority Element](https://leetcode.com/problems/majority-element/)

This problem can be trivially solved in O(N) time and space by just counting the elements:
```python
from collections import Counter

def majorityElement(nums: List[int]) -> int:
    return Counter(nums).most_common(1)[0][0]
```

A divide-and-conquer approach will potentially take O(nlogn) time and space; it's not the optimal solution, but this problem is good practice for the technique anyways:
- **Divide** the array
- **Conquer** subdivided elements in a straightforward manner; if len(arr) == 1, then the single element is the majority.
- **Combine** the results - at each combine step, we can do a linear time operation since we will do at most log(n) combinations; in this case, we can just do the linear time counting to determine which element is the maximum for that subarray.

In [ ]:
from collections import Counter

def majority(arr,lo,hi):
    if lo == hi:
        return arr[lo]
    mid = (hi+lo)//2
    lo_majority = majority(arr, lo, mid)
    hi_majority = majority(arr,mid+1,hi)
    
    # if both have same majority, then return it
    if lo_majority == hi_majority:
        return lo_majority
    # Otherwise, count the occurrences and return the best
    return Counter(arr[lo:hi+1]).most_common(1)[0][0]


cases = [
    ([3,3,4], 3),
    ([2,2,1,1,1], 1),
    ([1,2,1,2,1], 1),
    ([1,1,1,2], 1),
]

for arr, expected in cases:
    actual = majority(arr, 0, len(arr)-1) 
    assert actual == expected, f"{arr}: {expected} != {actual}"

## Example: [Different Ways to Add Parentheses](https://leetcode.com/problems/different-ways-to-add-parentheses/)

In: string representing arithmetic expression (+,-,* only)
Out: list of int values that can result when fully parenthesized 

Constraints: Not provided, other than operators 

Divide:
Conquer (base case): single operator and 2 ints
Linear combine: can evaluate an entire arithmetic expression

"2-1-1"
((2-1)-1) = 0 
(2-(1-1)) = 2

For n operators, there are n! orderings. Do we need to evaluate them all? Looks like it. 

- **Divide** - each possible operator can be split on
- **Conquer** - compute each single operator / two operand expression
- **Combine** - combine results 

In [22]:
from typing import List

def compute(string, lo, hi):
    possible_ways = []
    for i in range(lo, hi+1):
        if string[i] in "-+*":
            lefts = compute(string, lo, i-1)
            rights = compute(string, i+1, hi)
            for left in lefts:
                for right in rights:
                    possible_ways.append(f"({left}{string[i]}{right})")
    return possible_ways if possible_ways else [f"({string[lo:hi+1]})"]


class Solution:
    def diffWaysToCompute(self, string: str) -> List[int]:
        return [eval(way) for way in compute(string, 0, len(string)-1)]

s = Solution()
cases = [
    ("2-1-1", [0, 2]),
    ("2*3-4*5", [-34, -14, -10, -10, 10]),
    ("11", [11]),
    ("10+5", [15])
]
for string, expected in cases:
    actual = s.diffWaysToCompute(string)
    assert sorted(actual) == sorted(expected), f"{string}: {expected} != {actual}"

## Example: [ Search a 2D Matrix II](https://leetcode.com/problems/search-a-2d-matrix-ii/)

This problem is obviously an ideal case for binary search, but it can also be solved via divide and conquer too. We can have two possible `O(1)` base cases:
- In a 1x1 matrix, either the element matches the target or not.
- if an nxm matrix, `matrix[0][0] <= target <= matrix[n-1][m-1]` must hold or the target isn't in the matrix.

So armed with that, we can apply the following:
```
search_matrix(matrix, target, top_left, bottom_right):
  if top_left == bottom_right:
      return true if the element matches the target
  if not matrix[top_left] <= target <= matrix[bottom_right]:
      return false 
  otherwise:
    top_left = search_matrix(matrix, target, top_left_subquadrant)
    top_right = search_matrix(matrix, target, top_right_subquadrant)
    bot_left = search_matrix(matrix, target, bottom_left_subquadrant)
    bot_right = search_matrix(matrix, target, bottom_right_subquadrant)
    return top_left or top_right or bot_left or bot_right
```

### Recursively subdividing arrays considered harmful (or at least tricky)
In this approach, we risk `IndexError` in our subdivision - if we have a submatrix where x and y are equal and on the boundary, e.g. `(4,1), (4,2)`, then `mid = (max_y+min_y//2)+1` will be out of bounds. However, the correct way to subdivide `(4,1), (4,2)` is into two single element submatrixes containing only those points, so we can just use `min(mid, len(matrix[0])-1)`

In [4]:
from typing import List

def search_matrix(matrix, target, min_y, min_x, max_y, max_x):
    if (min_y, min_x) == (max_y, max_x):
        return matrix[min_y][min_x] == target

    if not matrix[min_y][min_x] <= target <= matrix[max_y][max_x]:
        return False
    
    mid_row = (max_y + min_y) // 2
    next_row = min(mid_row+1, len(matrix)-1)
    mid_col = (max_x + min_x) // 2
    next_col = min(mid_col+1, len(matrix[0])-1)

    
    top_left = search_matrix(matrix, target, min_y, min_x, mid_row, mid_col)
    top_right = search_matrix(matrix, target, min_y,  next_col, mid_row, max_x)
    bot_left = search_matrix(matrix, target, next_row, min_x, max_y, mid_col)
    bot_right = search_matrix(matrix, target, next_row, next_col, max_y, max_x)
    
    return top_left or top_right or bot_left or bot_right 

class Solution:
    def searchMatrix(self, matrix: List[List[int]], val: int) -> bool:
        return search_matrix(matrix, val, 0,0, len(matrix)-1, len(matrix[0])-1)

cases = [
    ([[1]], 1, True),
    ([
        [1,4,7,11,15],
        [2,5,8,12,19],
        [3,6,9,16,22],
        [10,13,14,17,24],
        [18,21,23,26,30]
    ], 20, False)
]
s = Solution()
for matrix, val, expected in cases:
    actual = s.searchMatrix(matrix, val)
    assert actual == expected, f"{matrix,val}: {expected} != {actual}"